In [1]:
import fundamentalanalysis as fa
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime
import statistics as st
from matplotlib.pyplot import figure

%matplotlib inline 

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [3]:
# api_key = '00ef9804fcde0edd93b1b4821ee2f06a' #job.chap@icloud.com
api_key = 'c81352430e2fe3c941faf0814227562b' #jobs.chaps@gmail.com
# api_key = 'a765d11740cccfb61177da0ad8699d1e' #job.chap@gmail.com

In [4]:
''' Колонки '''
# ticker = 'ATEX'
ticker = 'AAPL'



''' для фильтрации'''
filt_start_date = '1999'
filt_finish_date = '2022'


''' время '''
stime=f'{str(int(filt_start_date)-1)}-01-01'
ftime='2023-02-01'
period = '1mo'  # '1d' '1wk'

In [5]:
def get_data_from_ticker(tick, stime, ftime, interval=period):
    ticker = yf.Ticker(tick)

    df = ticker.history(start=stime, end=ftime, interval=interval)
    x = pd.DataFrame(df)
    x.rename(columns={"Close": tick}, inplace=True)
    z = x.drop(columns=["Open", "High", "Low", "Volume", "Dividends", "Stock Splits"])
    return z

df = get_data_from_ticker(ticker, stime, None, period)
df_market = get_data_from_ticker('^GSPC', stime, None, period)
df.tail(3)

,AAPL
Date,
2022-12-01 00:00:00-05:00,129.731918
2023-01-01 00:00:00-05:00,144.070023
2023-02-01 00:00:00-05:00,152.550003


In [6]:
def change_index(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        t = df_index[i].date()
        df_index[i] = t.strftime('%Y-%m-%d')
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

df = change_index(df)
df_market  = change_index(df_market)
df.tail(3)

,AAPL
Date_index,
2022-12-01,129.731918
2023-01-01,144.070023
2023-02-01,152.550003


In [7]:
''' добавляем цифру к году, чтобы видень данные как начало года, а не как факт '''
def change_index_plus_one(df):
    df_index = df.index.to_list()
    for i in range(len(df_index)):
        df_index[i] = str(int(df_index[i])+1)
    df['Date_index'] = df_index
    df = df.set_index('Date_index')
    return df

In [8]:
key_metrics = fa.key_metrics(ticker, api_key, period='annual')
df_key_metr = key_metrics.T
df_key_metr = change_index_plus_one(df_key_metr)
df_key_metr.head(3)

,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
Date_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023,FY,24.317273,6.154614,7.532763,6.872426,2.978793,3.124822,3.124822,3.124822,7.585118,2423475620080.514648,2519898620080.514648,24.282593,6.145837,19.839998,21.746324,47.826721,47.826721,6.390362,18.926968,20.629374,22.611547,0.041182,0.045985,1.952933,0.856354,0.724234,0.879356,40.749574,1.223921,0.006124,0.148703,0,0.066571,0,-0.087662,-0.027155,-0.964337,0.02292,20.801964,0.600871,0.282924,-12.679296,-18577000000,None,-166678000000,2.369533,56219000000,59439000000,5763000000,56.400205,104.685277,8.075698,6.471608,3.486641,45.197331,1.969589,-0.660337
2022,FY,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,3.777557,3.777557,7.626006,2482477060059.236816,2572256060059.236816,26.219656,6.786117,23.861253,26.706799,39.348186,39.348186,7.031538,20.889554,24.724198,27.672652,0.038139,0.037444,1.729371,0.820257,0.729104,1.074553,41.190548,1.098838,0.005828,0.152799,0,0.059904,0,-0.106548,-0.030302,-0.982364,0.021612,21.950838,0.483099,0.269742,-10.978415,9355000000,None,-153076000000,1.976843,51506000000,54763000000,6580000000,51.390969,93.851071,11.276593,7.102415,3.88914,32.367933,1.500713,-0.663722
2021,FY,15.820258,3.308587,4.64923,4.228014,5.241031,3.765477,3.765477,3.765477,6.64524,1996361344741.568848,2070781344741.568848,34.773151,7.272322,24.746031,27.211359,30.553901,30.553901,7.543418,25.558891,25.66851,28.225739,0.028758,0.036749,1.510078,0.798267,0.918539,1.363604,23.072746,1.405201,0.007053,0.245267,0,0.06831,0,-0.090599,-0.026625,-0.661089,0.024877,16.742586,0.303383,0.177256,-7.923629,38321000000,None,-114836000000,1.72081,44475500000,48529500000,5320500000,49.787534,91.04819,8.741883,7.331152,4.008866,41.753016,0.878664,-0.421217


In [9]:
df_key_metr = df_key_metr.loc[filt_finish_date:filt_start_date].copy(deep=True)
index_years = df_key_metr.index.to_list()

In [10]:
def get_price_from_df(df, list_dates):
    new_list = []
    for i in range(len(list_dates)):
        x = int(list_dates[i])
        # x += 1
        x = str(x)
        r = df.loc[f'{x}-01-01']
        r = float(r)
        new_list.append(r)
        
    return new_list
        
stock_year_prices = get_price_from_df(df, index_years)
stock_year_market_prices = get_price_from_df(df_market, index_years)

In [11]:
df_key_metr[ticker] = list(stock_year_prices)
df_key_metr['SNP'] = list(stock_year_market_prices)
df_key_metr.drop('period', axis=1, inplace=True)

In [12]:
def tf_all_columns(df):
    x = df.columns.to_list()
    for i in range(len(x)):
        v = df[x[i]].to_list()
        l = []
        for y in range(len(v)):
            try:
                # if y != len(v):
                #     vv = (v[y] / (v[y+1] / 100)) - 100
                #     l.append(vv)
                #     # print(vv)
                if y != len(v):
                    if v[y] < v[y+1]:
                        l.append('False')
                    elif v[y] > v[y+1]:
                        l.append('True')
                    else:
                        l.append('None')  
                else:
                    vv = 'None'
                    l.append(vv)
                    # print(vv)
            except:
                l.append(0)
        df[f'TF {x[i]}'] = l
    return df

df_with_tf = tf_all_columns(df_key_metr)

In [13]:
df_with_tf.head(2)

,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,shareholdersEquityPerShare,interestDebtPerShare,marketCap,enterpriseValue,peRatio,priceToSalesRatio,pocfratio,pfcfRatio,pbRatio,ptbRatio,evToSales,enterpriseValueOverEBITDA,evToOperatingCashFlow,evToFreeCashFlow,earningsYield,freeCashFlowYield,debtToEquity,debtToAssets,netDebtToEBITDA,currentRatio,interestCoverage,incomeQuality,dividendYield,payoutRatio,salesGeneralAndAdministrativeToRevenue,researchAndDdevelopementToRevenue,intangiblesToTotalAssets,capexToOperatingCashFlow,capexToRevenue,capexToDepreciation,stockBasedCompensationToRevenue,grahamNumber,roic,returnOnTangibleAssets,grahamNetNet,workingCapital,tangibleAssetValue,netCurrentAssetValue,investedCapital,averageReceivables,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare,AAPL,SNP,TF revenuePerShare,TF netIncomePerShare,TF operatingCashFlowPerShare,TF freeCashFlowPerShare,TF cashPerShare,TF bookValuePerShare,TF tangibleBookValuePerShare,TF shareholdersEquityPerShare,TF interestDebtPerShare,TF marketCap,TF enterpriseValue,TF peRatio,TF priceToSalesRatio,TF pocfratio,TF pfcfRatio,TF pbRatio,TF ptbRatio,TF evToSales,TF enterpriseValueOverEBITDA,TF evToOperatingCashFlow,TF evToFreeCashFlow,TF earningsYield,TF freeCashFlowYield,TF debtToEquity,TF debtToAssets,TF netDebtToEBITDA,TF currentRatio,TF interestCoverage,TF incomeQuality,TF dividendYield,TF payoutRatio,TF salesGeneralAndAdministrativeToRevenue,TF researchAndDdevelopementToRevenue,TF intangiblesToTotalAssets,TF capexToOperatingCashFlow,TF capexToRevenue,TF capexToDepreciation,TF stockBasedCompensationToRevenue,TF grahamNumber,TF roic,TF returnOnTangibleAssets,TF grahamNetNet,TF workingCapital,TF tangibleAssetValue,TF netCurrentAssetValue,TF investedCapital,TF averageReceivables,TF averagePayables,TF averageInventory,TF daysSalesOutstanding,TF daysPayablesOutstanding,TF daysOfInventoryOnHand,TF receivablesTurnover,TF payablesTurnover,TF inventoryTurnover,TF roe,TF capexPerShare,TF AAPL,TF SNP
Date_index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022,21.903541,5.669029,6.229346,5.565624,3.750553,3.777557,3.777557,3.777557,7.626006,2482477060059.236816,2572256060059.236816,26.219656,6.786117,23.861253,26.706799,39.348186,39.348186,7.031538,20.889554,24.724198,27.672652,0.038139,0.037444,1.729371,0.820257,0.729104,1.074553,41.190548,1.098838,0.005828,0.152799,0,0.059904,0,-0.106548,-0.030302,-0.982364,0.021612,21.950838,0.483099,0.269742,-10.978415,9355000000,None,-153076000000,1.976843,51506000000,54763000000,6580000000,51.390969,93.851071,11.276593,7.102415,3.88914,32.367933,1.500713,-0.663722,173.506546,4515.549805,True,True,True,True,False,True,True,True,True,True,True,False,False,False,False,True,True,False,False,False,False,True,True,True,True,False,False,True,False,False,False,None,False,None,False,False,False,False,True,True,True,False,False,0,False,True,True,True,True,True,True,True,False,False,False,True,False,True,True
2021,15.820258,3.308587,4.64923,4.228014,5.241031,3.765477,3.765477,3.765477,6.64524,1996361344741.568848,2070781344741.568848,34.773151,7.272322,24.746031,27.211359,30.553901,30.553901,7.543418,25.558891,25.66851,28.225739,0.028758,0.036749,1.510078,0.798267,0.918539,1.363604,23.072746,1.405201,0.007053,0.245267,0,0.06831,0,-0.090599,-0.026625,-0.661089,0.024877,16.742586,0.303383,0.177256,-7.923629,38321000000,None,-114836000000,1.72081,44475500000,48529500000,5320500000,49.787534,91.04819,8.741883,7.331152,4.008866,41.753016,0.878664,-0.421217,130.195908,3714.239990,True,True,True,True,False,False,False,False,True,True,True,True,True,True,True,True,True,True,True,True,True,False,False,True,True,True,False,True,True,False,False,None,True,None,True,True,True,Tru

In [14]:
df_some_columns = df_with_tf[['TF debtToAssets', f'TF {ticker}']]
df_some_columns 

,TF debtToAssets,TF AAPL
Date_index,,
2022,True,True
2021,True,True
2020,True,True
2019,True,True
2018,True,True
2017,True,True
2016,True,False
2015,True,True
2014,True,True
